# Assessing the Stiffness of a Network

Reaction networks with a wide range of rate timescales can be stiff, generally requiring implicit integrators.  Sometimes there might be only a single rate that is responsible for the stiffness, and in this case, it might be possible to do an approximation to alleviate some of the stiffness.

Here we look at how to characterize the stiffness of the rate.

For a network of the form:

$$\frac{d{\bf Y}}{dt} = {\bf f}({\bf Y})$$

We construct the Jacobian as:

$${\bf J} = \frac{\partial \dot{\bf Y}}{\partial {\bf Y}}$$

If we compute the eigenvalues $\lambda_i$, of ${\bf J}$, then a stiff reaction network is characterized by:

$$\frac{\max_i\{|\lambda_i|\}}{\min_i\{|\lambda_i|\}} \gg 1$$

and related concept is the [spectral radius](https://en.wikipedia.org/wiki/Spectral_radius):

$$\rho({\bf J}) = \max_i\{|\lambda_i|\}$$

Some integrators (like Runge-Kutta-Chebyshev) use this to determine the number of stages needed for each integration
step.

In pynucastro, the spectral radius of a network can be computed using {py:func}`spectral_radius <pynucastro.networks.rate_collection.RateCollection.spectral_radius>`.

In [ ]:
import pynucastro as pyna

## H burning network example

We'll create a network with the nuclei needed for various p-p and CNO burning sequences, as well as breakout.  

In [ ]:
all_nuclei = ["p", "h2", "he3", "he4",
              "li6", "li7", "be7", "be8", "b8",
              "c12", "c13", "n13", "n14", "n15",
              "o14", "o15", "o16", "o17", "o18",
              "f17", "f18", "f19",
              "ne18", "ne19", "ne20", "ne21"]


In [ ]:
rl = pyna.ReacLibLibrary()

In [ ]:
lib = rl.linking_nuclei(all_nuclei, with_reverse=False)
net = pyna.RateCollection(libraries=lib)

In [ ]:
fig = net.plot(hide_xalpha=True, hide_xp=True,
               node_size=500, node_font_size=10,
               size=(800, 800))

## Spectral radius

Now we'll choose conditions appropriate to the base of the convective H-burning envelope in a classical nova.

```{important}
The spectral radius will depend somewhat on the composition of the trace nuclei, so they should be varied in any analysis to cover what would be expected in a simulation.
```

In [ ]:
rho = 1700
T = 7.e7
comp = pyna.Composition(net.unique_nuclei)
comp.set_all(1.e-6)
comp.set_nuc("h1", 0.7)
comp.set_nuc("he4", 0.28)
comp.set_nuc("n14", 0.02)
comp.normalize()

In [ ]:
fig = comp.plot(trace_threshold=0.01)

If we evaluate the spectral radius, we see that it is reasonably large.

In [ ]:
sprad = net.spectral_radius(rho, T, comp)
print(f"{sprad:12.7g}")

## Alleviating stiffness

We can try to understand which rate is responble for making the spectral radius large by looping over each rate and recomputing the spectral radius without that rate included.

In [ ]:
for r in net.get_rates():
    print(f"{r.fname:35} ({r.Q:6.2f}):  {net.spectral_radius(rho, T, comp, exclude_rates=[r]):9.2f}")

F17_to_O17_reaclib                  (  2.76):  345703.34
F18_to_O18_reaclib                  (  1.66):  345703.34


Ne18_to_F18_reaclib                 (  4.44):  345703.34
Ne19_to_F19_reaclib                 (  3.24):  345703.34
B8_to_He4_He4_reaclib               ( 18.07):  345703.34
p_p_to_d_reaclib_bet_pos            (  1.44):  345703.34
p_p_to_d_reaclib_electron_capture   (  1.44):  345703.34


p_d_to_He3_reaclib                  (  5.49):  345703.34


d_d_to_He4_reaclib                  ( 23.85):  345703.34
He4_d_to_Li6_reaclib                (  1.47):  345703.34
p_He3_to_He4_reaclib                ( 19.80):  345703.34
He4_He3_to_Be7_reaclib              (  1.59):  345703.34
p_Li6_to_Be7_reaclib                (  5.61):  345691.74
p_Be7_to_B8_reaclib                 (  0.14):  345703.34


p_C12_to_N13_reaclib                (  1.94):  345703.34


...

p_Li7_to_He4_He4_reaclib            ( 17.35):  345703.34


C12_C12_to_He4_Ne20_reaclib         (  4.62):  345703.34
He4_N13_to_p_O16_reaclib            (  5.22):  345703.34
p_N15_to_He4_C12_reaclib            (  4.97):  345703.34
He4_O14_to_p_F17_reaclib            (  1.19):  345703.34
p_O17_to_He4_N14_reaclib            (  1.19):  345703.34
p_O18_to_He4_N15_reaclib            (  3.98):  345703.34


p_F18_to_He4_O15_reaclib            (  2.88):  345703.34


He4_F18_to_p_Ne21_reaclib           (  1.74):  345703.34
p_F19_to_He4_O16_reaclib            (  8.11):  345703.34
p_Ne20_to_He4_F17_reaclib           ( -4.13):  345703.34
He3_He3_to_p_p_He4_reaclib          ( 12.86):  345703.34
d_Be7_to_p_He4_He4_reaclib          ( 16.77):  345703.34
He3_Be7_to_p_p_He4_He4_reaclib      ( 11.27):  345703.34


He4_He4_He4_to_C12_reaclib          (  7.28):  345703.34


From here we see that it is just the ${}^6\mathrm{Li}(\mathrm{p},{}^3\mathrm{He}){}^4\mathrm{He}$ rate that is responsible.  Removing that rate drops the spectral radius by almost $50\times$.

We can look at all the rates that involve ${}^6\mathrm{Li}$ and see how fast they are:

In [ ]:
rates = net.evaluate_rates(rho, T, comp)

In [ ]:
for r in net.get_rates():
    if pyna.Nucleus("li6") in r.reactants + r.products:
        print(r, rates[r])

This shows that the same rate responsible for increasing the spectral radius is also the fastest rate involving ${}^6\mathrm{Li}$.

At this stage, there are a few paths that can be taken.

Based on the rate strengths, we might not ever need to consider ${}^4\mathrm{He}(\mathrm{d}, \gamma){}^6\mathrm{Li}$, which removes the production of ${}^6\mathrm{Li}$ from the network completely, so we could just remove that nucleus.  This would be reasonable if there were something else that consumes $\mathrm{d}$ much faster.  Let's see:

In [ ]:
for r in net.get_rates():
    if pyna.Nucleus("d") in r.reactants:
        print(r, rates[r])

All of these rates are much faster than ${}^4\mathrm{He}(\mathrm{d}, \gamma){}^6\mathrm{Li}$, so it does seem reasonable that we can just remove that nucleus.

Alternately, we can approximate out ${}^6\mathrm{Li}$ by assuming equilibrium of the 4 rates listed above, setting:

$$\frac{dY({}^6\mathrm{Li})}{dt} = 0$$

and then write effective rates that link to ${}^7\mathrm{Li}$ and ${}^7\mathrm{Be}$.